In [ ]:
import sys, os
import time
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022'
sys.path.insert(1, src_dir_)

import numpy as np
import matplotlib.pyplot as plt

from src.util import read_pickle, ConsoleLogger, generate_node_cargo_size_change
from pdpt_ini_sol import initialization_pdotw
from src.pdotw_mip import eval_pdotw_sol, group_cycle_truck, pdotw_mip_gurobi, postprocess_solution_pdotw


In [ ]:
dir_ = '/home/tan/Documents/GitHub/pdpt_2022'

case_num  = 1
verbose = 0

In [ ]:
def test():
        pdpt_ins_file = os.path.join(dir_,'data/', f'case{case_num}.pkl')
        pdpt_ins = read_pickle(pdpt_ins_file, verbose = verbose-2) 

        edge = pdpt_ins['edge_shortest']
        truck_list = pdpt_ins['truck']  #truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
        cargo_list = pdpt_ins['cargo']  #cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']

                        #dis(truck_origin, parcel_origin) +  dis(parcel_origin, parcel_dest) +  dis(parcel_dest, truck_dest)
        cost_to_deliver = np.array([ [sum([edge[(truck_value[0], cargo_value[-2])],\
                                edge[(cargo_value[-2], cargo_value[-1])],\
                                edge[(cargo_value[-1], truck_value[1])] ])
                                        for cargo_key, cargo_value in cargo_list.items()]
                                                for truck_key, truck_value in truck_list.items()])
        assert cost_to_deliver.shape == (len(truck_list.keys()), len(cargo_list.keys()))

        truck_key = list(truck_list.keys())[0]

        idx = list(truck_list.keys()).index(truck_key)

        print(idx)

        truck_cost_to_deliver_parcels = cost_to_deliver[idx]

        print(truck_cost_to_deliver_parcels)

        idx_sorted = np.argsort(truck_cost_to_deliver_parcels, axis=-1, kind='quicksort', order=None)

        print(truck_cost_to_deliver_parcels[idx_sorted])

        cargo_keys = list(cargo_list.keys())

In [ ]:
def solve_pdotw_mip(ins,  # dict contains the data of pdpt instance,
                    path_, # file where all data of pdotw solutions are saved
                    greedy_initialization = False,
                    verbose = 0):  


    res, truck_keys_shuffle, selected_truck, selected_cargo = initialization_pdotw(ins, greedy_initialization, verbose)



    # load data from ins
    truck = ins['truck']
    cargo = ins['cargo']
    cargo_key_list = list(cargo.keys())
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']

    min_parcels = int(0.2*len(cargo.keys()))

    cost_to_deliver = np.array([ [sum([edge_shortest[(truck_value[0], cargo_value[-2])],\
                         edge_shortest[(cargo_value[-2], cargo_value[-1])],\
                         edge_shortest[(cargo_value[-1], truck_value[1])] ])
                                    for cargo_key, cargo_value in cargo.items()]
                                            for truck_key, truck_value in truck.items()])

    created_truck_yCycle_total, created_truck_nCycle_total, created_truck_all_total, node_list_truck_hubs_total = res[0]
    x_sol_total, y_sol_total, S_sol_total, D_sol_total, A_sol_total, Sb_sol_total, Db_sol_total, Ab_sol_total = res[1]
    cost_cargo_size_value_total,  cost_cargo_number_value_total, cost_travel_value_total, cost_deviation_value_total = res[2]
    truck_used_total, truck_route, cargo_delivered_total, cargo_undelivered_total, lb_truck, cargo_route, truck_per_cargo, cargo_in_truck = res[3]

    # edge_shortest, path_shortest = replace_edge_by_shortest_length_nx(nodes, edges)
    # single_truck_deviation = calculate_single_truck_deviation(truck, cargo, edge_shortest)
    time_start_all = time.time()

    remaining_cargo = selected_cargo.copy()
    
    for truck_key in truck_keys_shuffle:
        print(f'len(remaining_cargo): {len(remaining_cargo.keys())}')
        selected_cargo = {}

        idx = list(truck.keys()).index(truck_key)
        truck_cost_to_deliver_parcels = cost_to_deliver[idx]
        idx_sorted = np.argsort(truck_cost_to_deliver_parcels, axis=-1, kind='quicksort', order=None)


        if len(remaining_cargo.keys()) < min_parcels:
            selected_cargo = remaining_cargo.copy()
        else:
            cargo_idx = 0
            while len(selected_cargo.keys()) < min_parcels and cargo_idx < len(cargo_key_list):
                cargo_key = cargo_key_list[idx_sorted[cargo_idx]]
                if cargo_key in remaining_cargo.keys():
                    selected_cargo[cargo_key] = remaining_cargo[cargo_key]
                cargo_idx += 1

        print(f'len(selected_cargo): {len(selected_cargo.keys())}')


        if verbose >0:
            print(f'========= START [PDOTW with truck {truck_key}] ========= ')

        tv = selected_truck[truck_key]
        
        created_truck = {}
        node_list_truck_hubs = {}
        created_truck[truck_key] = tv
        
        # nodes in the cluster
        # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
        # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']

        selected_node = []
        for v in selected_cargo.values():
            if v[3] not in selected_node:
                selected_node.append(v[3])
            if v[4] not in selected_node:
                selected_node.append(v[4])
        if tv[0] not in selected_node:
            selected_node.append(tv[0])
        if tv[1] not in selected_node:
            selected_node.append(tv[1])
        
        # edges in the cluster
        selected_edge = {}
        for i in selected_node:
            for j in selected_node:
                selected_edge[(i,j)] = edge_shortest[(i,j)]
        
        node_list_truck_hubs[truck_key] = selected_node.copy()
        node_list_truck_hubs_total[truck_key] = selected_node.copy()
        assert len(created_truck) == len(node_list_truck_hubs), "Inconsistent truck numbers"

        if verbose >0:
            print(f'+++ Preprocess data to instantiate a PDOTW MIP')
        if verbose > 2:
            print(f'    [selected_cargo] size: {len(selected_cargo)}')
            for key, value in selected_cargo.items():
                print(f'        {key, value}')
            print(f'    [created_truck] size: {len(created_truck)}')
            for key, value in created_truck.items():
                print(f'       {key, value}')
            print(f'    [node_list_truck_hubs] size: {len(node_list_truck_hubs)}')
            for key, value in node_list_truck_hubs.items():
                print(f'       {key, value}')
        
        ### Need to update node_cargo_size_change 
        node_cargo_size_change = \
        generate_node_cargo_size_change(selected_node, selected_cargo)

        ### group cycle and non-cycle trucks
        created_truck_yCycle, created_truck_nCycle, created_truck_all = \
        group_cycle_truck(created_truck)  
        
        if verbose > 2:
            print('    [created_truck_yCycle]', created_truck_yCycle)
            print('    [created_truck_nCycle]', created_truck_nCycle)

        if verbose > 2: 
            print('    [The created_truck_yCycle]')  
        for key, value in created_truck_yCycle.items():
            if verbose > 2: 
                print(f'       {key, value}')
            created_truck_yCycle_total[key] = value
        if verbose > 2: 
            print('    [The created_truck_nCycle]') 
        for key, value in created_truck_nCycle.items():
            if verbose > 2: 
                print(f'       {key, value}')
            created_truck_nCycle_total[key] = value
        if verbose > 2: 
            print('    [The created_truck_all]') 
        for key, value in created_truck_all.items():
            if verbose > 2: 
                print(f'       {key, value}')
            created_truck_all_total[key] = value

        if verbose >0:
            print(f'+++ Solve PDOTW MIP in Gurobi')
        ### use gurobi to solve the GROW origin PDPTW
        # Note. the pdotw_mip_gurobi function is desgined to take the same arguments as pdpt function
        # but some parameters
        gurobi_log_file = path_ + f'_gurobi/truck{truck_key}.log'

        obj_val_MP, runtime_MP, \
        x_sol, _, y_sol, S_sol, D_sol, A_sol, \
        Sb_sol, Db_sol, Ab_sol, \
        cost_cargo_size_value, cost_cargo_number_value, \
        cost_travel_value, cost_deviation_value\
        = pdotw_mip_gurobi(constant, 
        selected_cargo, single_truck_deviation,
        created_truck_yCycle, created_truck_nCycle, created_truck_all,
        node_list_truck_hubs, selected_edge, node_cargo_size_change,
        100, gurobi_log_file, verbose = 1)

        # Index k for truck is pre-defined
        #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
        #y_sol: y^k_r, if parcel r is carried by truck k
        #s_sol: x^k_i, total size of cargos on truck k at node i
        #D_sol: D^k_i, depature time of truck k at node i
        #A_sol: A^k_i, arrival time of truck k at node i


        ### if origin stage subproblem for the current truck is feasible
        if obj_val_MP >= 0:
            if verbose >0:
                print(f'+++ Postprocee Gurobi solution if a feasible solution is found')
            cost_cargo_size_value_total[truck_key] = cost_cargo_size_value
            cost_cargo_number_value_total[truck_key] = cost_cargo_number_value
            cost_travel_value_total[truck_key] = cost_travel_value
            cost_deviation_value_total[truck_key] = cost_deviation_value

            for key, value in x_sol.items():
                x_sol_total[key] = value
            for key, value in y_sol.items():
                y_sol_total[key] = value
            for key, value in S_sol.items():
                S_sol_total[key] = value
            for key, value in D_sol.items():
                D_sol_total[key] = value
            for key, value in A_sol.items():
                A_sol_total[key] = value
            for key, value in Sb_sol.items():
                Sb_sol_total[key] = value
            for key, value in Db_sol.items():
                Db_sol_total[key] = value
            for key, value in Ab_sol.items():
                Ab_sol_total[key] = value

            ### post-process the solution
            truck_used, cargo_delivered, cargo_undelivered, \
            cargo_truck_total, cargo_in_truck = \
            postprocess_solution_pdotw(cargo, truck, 
            selected_cargo, created_truck_all,
            node_list_truck_hubs, 
            x_sol, y_sol, truck_route, cargo_route, verbose = 1)
                    

            for truck_ in truck_used:
                if truck_ not in truck_used_total:
                    truck_used_total.append(truck_)

            for c_key, c_value in cargo_delivered.items():
                cargo_delivered_total[c_key] = c_value

            for c_key, c_value in cargo_undelivered.items():
                cargo_undelivered_total[c_key] = c_value

            for c_key, c_value in cargo_truck_total.items():
                if v != -1:
                    truck_per_cargo[c_key] = c_value
            for t_key, t_value in cargo_in_truck.items():
                cargo_in_truck[t_key] = t_value.copy()

            ### Remove carried cargo ######
            for key, value in y_sol.items():
                if value == 1:
                    del remaining_cargo[key[1]]

        print(f'========= END [PDOTW with truck {truck_key}] ========= \n')

    truck_cost, travel_cost = eval_pdotw_sol(constant, edge_shortest, truck_used_total, truck_route)

    res = {'MIP': {'x_sol': x_sol_total,
                   'y_sol': y_sol_total,
                   'S_sol': S_sol_total,
                   'D_sol': D_sol_total,
                   'A_sol': A_sol_total,
                   'Sb_sol': Sb_sol_total,
                   'Db_sol': Db_sol_total,
                   'Ab_sol': Ab_sol_total,
                  },
           'route': {'truck_yCycle':list(created_truck_yCycle_total.keys()),
                     'used_truck': truck_used_total,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                    },
           'cost': {'truck_cost' : truck_cost,
                    'travel_cost' : travel_cost,
                    },
          }

    if verbose > 1:
        print('+++ Summary of the initial solution')
        print('\nThe final length of remaining_cargo is:', len(remaining_cargo))
        removed_cargo = []
        for c in remaining_cargo.keys():
            removed_cargo.append(c)

        print('\nThe truck_used_total:', len(truck_used_total))
        print(truck_used_total)

        assert len(truck_used_total) <= len(truck) , \
        "len(truck_used_total) > len(truck)"

        print('\nThe truck_route:', len(truck_route))
        for t, v in truck_route.items():
            print(t, v)
        assert len(truck) == len(truck_route), "len(truck) != len(truck_route)"

        print('\nThe cargo_route:', len(cargo_route))
        for c, v in cargo_route.items():
            print(c, v)
        assert len(cargo) == len(cargo_route), "len(cargo) != len(cargo_route)"

        if verbose>2:
            print('\nThe cargo_delivered_total:', len(cargo_delivered_total))
            print([cargo_key for cargo_key in cargo_delivered_total.keys()])
            
            print('\nThe cargo_undelivered_total:', len(cargo_undelivered_total))
            print([cargo_key for cargo_key in cargo_undelivered_total.keys()])

            print('\nThe truck_per_cargo:', len(truck_per_cargo))
            for t, v in truck_per_cargo.items():
                print(t, v)
            assert len(truck_per_cargo) == len(cargo), \
            "len(truck_per_cargo) != len(cargo)"

            print('\nThe cargo_in_truck:', len(cargo_in_truck))
            for t, v in cargo_in_truck.items():
                print(t, v)

        print('\nThe removed_cargos are:', len(removed_cargo))
        print(removed_cargo)

        time_end_all = time.time()
        print('\nThe total runtime is:', time_end_all - time_start_all)


    return res

In [ ]:
logfile = os.path.join(dir_, 'notebook','WIP_pdotw_min_deviation.log')

# ConsoleLogger.start(logfile, mode='w', time_format='[%m-%d %H:%M:%S]')
pdpt_ins_file = os.path.join(dir_,'data/', f'case{case_num}.pkl')
pdpt_ins = read_pickle(pdpt_ins_file, verbose = verbose-2) 
path_ = dir_+'/out/case' + str(case_num) 
res = solve_pdotw_mip(pdpt_ins, path_, False, verbose = 1)

# ConsoleLogger.stop()
